The purpose of this notebook is to create a data structure that allows for quick lookup of harmonic equivalence, rather than determining it by rotating matrices constantly.

My idea is to have a dictionary of dictionaries:
    each top-level key is a chord name (e.g. 'C' or 'Amin')
    the value for each top-level key is a dictionary object, whose keys are equivalent chord names, and whose values are the distance from the top-level key (measured in semitones)

Example entry for 'C' would be a dictionary with the following key,value pairs:
    'Cs' : 1,
    'D' : 2, 
    'Ds' : 3,
    'Eb' : 3,
    'E' : 4,
    etc.

In [36]:
# importing basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
from collections import Counter, deque

data_folder_path = '../../data/'

In [37]:
# Read the mapping CSV file
chord_relations = pd.read_csv(data_folder_path + 'chords_mapping.csv')

# Create a dictionary with keys the "chords" and values the "degrees"
chord_degrees = dict(zip(chord_relations['Chords'], chord_relations['Degrees']))
for key, value in chord_degrees.items():
    chord_degrees[key] = ast.literal_eval(value)
    
# full list of chords from the chords_mapping csv
known_chords = list(chord_degrees.keys())

In [38]:
print(chord_degrees['C'])
print(chord_degrees['Cs'])
print(chord_degrees['Db'])
print(chord_degrees['D'])

[1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]


In [39]:
# method to transpose a chord in vector format
def transpose_chord_up(chord_vector, num_semitones):
    d = deque(chord_vector)
    d.rotate(num_semitones)
    return(list(d))

# method to return true if chord_1 and chord_2 are just tranposed versions of each other
def is_harmonic_equivalent(chord_1, chord_2, get_distance = False):
    # if get_distance is true, then also return the number of semitones from chord_1 to chord_2 (if they are equivalent, otherwise that second return value will be None)
    
    # if they have different numbers of notes, then we don't have to check if 
    # any of the transpositions are equal
    if sum(chord_1) != sum(chord_2):
        if get_distance:
            return (False, None)
        else:
            return False

    # if they have the same number of notes, just rotate through the 12 possible transpositions to check
    for i in range(12):
        if np.array_equal(chord_2, transpose_chord_up(chord_1, i)):
            if get_distance:
                return (True, i)
            else:
                return True

    # return default value of false if no equivalent rotation was found
    if get_distance:
        return (False, None)
    else: 
        return False

C = chord_degrees['C']
D = chord_degrees['D']
E = chord_degrees['E']
G = chord_degrees['G']
assert(is_harmonic_equivalent(C,D))
assert(is_harmonic_equivalent(C,E))
assert(is_harmonic_equivalent(D,E))
assert(is_harmonic_equivalent(C,G))

Cmaj7 = chord_degrees['Cmaj7']
Dmaj7 = chord_degrees['Dmaj7']
assert(is_harmonic_equivalent(Cmaj7,Dmaj7))

equiv, dist = is_harmonic_equivalent(chord_1 = C, chord_2 = D, get_distance = True)
assert(equiv)
assert(dist == 2)

Time to build the equivalence dictionary.

In [41]:
# method to build one of the low-level dictionaries for a given chord
def single_chord_harmonic_equivalence_dictionary(my_chord):
    chord_1 = chord_degrees[my_chord]
    my_dict = {}
    for c in chord_degrees.keys():
        chord_2 = chord_degrees[c]
        my_bool, num_semitones = is_harmonic_equivalent(chord_1, chord_2, get_distance = True)
        if my_bool:
            my_dict[c] = num_semitones
    return my_dict

sing_chord_harmonic_equivalence_dictionary('C')

{'C': 0,
 'Cs': 1,
 'D': 2,
 'Ds': 3,
 'E': 4,
 'Es': 5,
 'Fs': 6,
 'G': 7,
 'Gs': 8,
 'A': 9,
 'As': 10,
 'B': 11,
 'Db': 1,
 'Eb': 3,
 'F': 5,
 'Gb': 6,
 'Ab': 8,
 'Bb': 10}

In [42]:
equiv_dict = {}
i=0
print("Compiling equivalence dictionary for a total of " + str(len(chord_degrees.keys())) + " chords.")
for c in chord_degrees.keys():
    equiv_dict[c] = single_chord_harmonic_equivalence_dictionary(c)
    i+=1
    if i % 200 == 0:
        print("Completed dictionary for " + str(i) + " chords.")

print(equiv_dict['C'])
print(equiv_dict['G'])
print(equiv_dict['Amin'])

Compiling equivalence dictionary for a total of 2793 chords.
Completed dictionary for 200 chords.
Completed dictionary for 400 chords.
Completed dictionary for 600 chords.
Completed dictionary for 800 chords.
Completed dictionary for 1000 chords.
Completed dictionary for 1200 chords.
Completed dictionary for 1400 chords.
Completed dictionary for 1600 chords.
Completed dictionary for 1800 chords.
Completed dictionary for 2000 chords.
Completed dictionary for 2200 chords.
Completed dictionary for 2400 chords.
Completed dictionary for 2600 chords.
{'C': 0, 'Cs': 1, 'D': 2, 'Ds': 3, 'E': 4, 'Es': 5, 'Fs': 6, 'G': 7, 'Gs': 8, 'A': 9, 'As': 10, 'B': 11, 'Db': 1, 'Eb': 3, 'F': 5, 'Gb': 6, 'Ab': 8, 'Bb': 10}
{'C': 5, 'Cs': 6, 'D': 7, 'Ds': 8, 'E': 9, 'Es': 10, 'Fs': 11, 'G': 0, 'Gs': 1, 'A': 2, 'As': 3, 'B': 4, 'Db': 6, 'Eb': 8, 'F': 10, 'Gb': 11, 'Ab': 1, 'Bb': 3}
{'Cmin': 3, 'Csmin': 4, 'Dmin': 5, 'Dsmin': 6, 'Emin': 7, 'Esmin': 8, 'Fsmin': 9, 'Gmin': 10, 'Gsmin': 11, 'Amin': 0, 'Asmin': 1, 

In [43]:
# if the two input chords are harmonically equivalent, return (True, num_semitones) where num_semitones is the distance from n_gram_1 (up) to n_gram_2
# otherwise, return (False, None)
def compare_chords(chord_1, chord_2):
    if chord_2 in equiv_dict[chord_1]:
        return (True, equiv_dict[chord_1][chord_2])
    else:
        return (False, None)

assert(compare_chords('C','D') == (True, 2))
assert(compare_chords('C','E') == (True, 4))
assert(compare_chords('C','Amin') == (False, None))

In [44]:
# if the two input n_grams are harmonically equivalent, return (True, num_semitones) where num_semitones is the distance from n_gram_1 (up) to n_gram_2
# otherwise, return (False, None)
def compare_n_grams(n_gram_1, n_gram_2):
    list_1 = n_gram_1.split(',')
    list_2 = n_gram_2.split(',')

    # if they aren't the same length, we don't have to check anything
    if len(list_1) != len(list_2):
        return (False, None)

    # now we can assume they have the same length
    comparison = [compare_chords(list_1[i], list_2[i]) for i in range(len(list_1))]

    # if any pairs are not the same, return False
    for c in comparison:
        if not c[0]:
            return (False, None)

    # now we can assume every respective pair is equivalent, but we still need all of the distances to match
    dist_0 = comparison[0][1]
    for c in comparison:
        if c[1] != dist_0:
            return (False, None)

    return (True, dist_0)

print(compare_n_grams('C,D,E','F,G,A'))
print(compare_n_grams('C,D,E','F,G,B'))

(True, 5)
(False, None)


In [45]:
# return true/false depending on if a song contains a harmonically equivalent n_gram to the input n_gram
# new version of this, making use of the equivalence dictionary for lookups rather than doing calculations every time
def contains_n_gram(song, n_gram):
    # assumption: input song is a comma-separated string of chord names
    # assumption: input n_gram is a comma-separated string of chord names

    # skip ahead and return true if the raw version is the song
    if n_gram in song:
        return True

    # split up the song and n_gram into lists of strings of single chords
    song_as_list = song.split(',')
    song_length = len(song_as_list)
    n_gram_as_list = n_gram.split(',')
    n = len(n_gram_as_list)

    for i in range(0,song_length - n):
        song_n_gram = ','.join(song_as_list[i:i+n])
        is_same, dist = compare_n_grams(n_gram, song_n_gram)
        #if is_same:
        #    return True
        print(song_n_gram)
        print(is_same)
        print()

assert(contains_n_gram('A,B,C,D,E,F,G','C,D'))
assert(contains_n_gram('A,B,C,D,E,F,G','F,G'))

In [46]:
import json
with open(data_folder_path + 'harmonic_equivalence_dictionary.json','w') as f:
    json.dump(obj = equiv_dict, fp = f)